In [ ]:
!pip install pandas-ta

In [ ]:
#Librerías a utilizar
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

In [ ]:
#Definir Ticker
Ticker='CEMEXCPO.MX'

#Definir la comision de Compra / Venta
Comision=0.0025 #Comisión de 0.25% por compra o venta de casa de bolsa

In [ ]:
#Definición de funciones a utilizar

#Función de extracción de información de Yahoo Finance de 1 mes cada 60 minutos
def get_data(symbol: str):
    data=yf.download(tickers=symbol, period='44d', interval='60m')
    data.reset_index(inplace=True)
    return data

# Función para calcular el Promedio Móvil
def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Función para determinar la tendencia - equilibrada
def determine_trend(data):
    if data['SMA_10']>data['SMA_20']>data['SMA_30']:
        return 2 # Tendencia de Alza
    elif data['SMA_10']<data['SMA_20']<data['SMA_30']:
        return 1 # Tendencia de Baja
    else:
        return 0 # Sin tendencia definida

In [ ]:
#Obtener la información de Yahoo Finance y renombrar columnas
data=get_data(Ticker)
data=data.set_axis(['Datetime','Close','High','Low','Open','Volume'], axis=1)

#Calcular los promedios moviles para los ultimos 10, 20 y 30  valores de la columa 'Close' y se agregan al data frame
data['SMA_10']=calculate_sma(data,10)
data['SMA_20']=calculate_sma(data,20)
data['SMA_30']=calculate_sma(data,30)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Determinar la tendencia en una nueva columna del dataframe
data['Tendencia']=data.apply(determine_trend,axis=1)

In [ ]:
# Determinar en que periodos hubo tendencia de alza y de baja - se guarda en un nuevo dataframe y lo reindexamos
df=data[data['Tendencia']!=0]
df=df.reset_index()
df

,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia
0,32,2025-01-20 16:30:00+00:00,11.27,11.28,11.18,11.19,532439,11.290,11.2485,11.238333,2
1,33,2025-01-20 17:30:00+00:00,11.23,11.28,11.21,11.26,235870,11.288,11.2570,11.226667,2
2,34,2025-01-20 18:30:00+00:00,11.25,11.26,11.21,11.23,189878,11.287,11.2670,11.214667,2
3,41,2025-01-22 15:30:00+00:00,11.48,11.53,11.39,11.42,1857082,11.311,11.3015,11.261667,2
4,42,2025-01-22 16:30:00+00:00,11.52,11.55,11.44,11.49,2376311,11.336,11.3130,11.277667,2
...,...,...,...,...,...,...,...,...,...,...,...
126,217,2025-03-12 16:30:00+00:00,12.08,12.08,12.01,12.07,635977,12.074,12.0855,12.288667,1
127,224,2025-03-13 16:30:00+00:00,11.92,12.07,11.91,11.99,1995905,12.043,12.0550,12.102667,1
128,225,2025-03-13 17:30:00+00:00,11.87,11.92,11.80,11.92,2223493,12.026,12.0455,12.079333,1
129,226,2025-03-13 18:30:00+00:00,11.84,11.87,11.77,11.85,1380225,12.003,12.0340,12.061667,1


In [ ]:
# Determinar la señal de Compra o Venta y definir el precio de dicha señal
df.loc[0,'Señal']=df.loc[0,'Tendencia']
for i in range(1,len(df)):
    if df.loc[i,'Tendencia']==df.loc[i-1,'Tendencia']:
        df.loc[i,'Señal']=0
    else:
        df.loc[i,'Señal']=df.loc[i,'Tendencia']

In [ ]:
#Corregir la primera fila, en caso de que la primera señal, sea una señal de venta
if df.loc[0,'Señal']==1:
    df.loc[0,'Señal']=0
else:
    df.loc[0,'Señal']=2

In [ ]:
#Quitar todos los ceros del dataframe con la columna 'Señal'
df=df[df['Señal']!=0]

#Reindexar finalmente el dataframe
df_Final=df.reset_index()
df_Final

,level_0,index,Datetime,Close,High,Low,Open,Volume,SMA_10,SMA_20,SMA_30,Tendencia,Señal
0,0,32,2025-01-20 16:30:00+00:00,11.27,11.28,11.18,11.19,532439,11.290,11.2485,11.238333,2,2.0
1,69,120,2025-02-14 14:30:00+00:00,13.47,13.73,13.46,13.62,1648215,13.533,13.5540,13.569000,1,1.0
2,70,130,2025-02-18 14:30:00+00:00,13.60,13.78,13.50,13.78,2578965,13.646,13.5895,13.584667,2,2.0
3,79,148,2025-02-21 17:30:00+00:00,13.39,13.55,13.38,13.53,2154118,13.564,13.5925,13.599333,1,1.0


In [ ]:
#Calcular el rendimiento de la estrategia
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Señal']==1:
        df_Final.loc[j,'Rendimiento']=df_Final.loc[j,'Close']*(1-Comision*1.16)/(df_Final.loc[j-1,'Close']*(1+Comision*1.16))-1
    else:
        df_Final.loc[j,'Rendimiento']=0

In [ ]:
#Renombrar la columna 'Señal'
for j in range(0,len(df_Final)):
    if df_Final.loc[j,'Tendencia']==2:
       df_Final.loc[j,'Señal']= "Compra"
    else:
        df_Final.loc[j,'Señal']="Venta"

<ipython-input-26-46ab7321370d>:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Compra' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_Final.loc[j,'Señal']= "Compra"


In [ ]:
#Generar un listado de columnas para mi resumen del Dataframe
colunmas=['index','Datetime','Close','Señal','Rendimiento']

#Seleccionar las columnas del dataframe 'df_Final'
df_Resumen=df_Final[colunmas]
df_Resumen


,index,Datetime,Close,Señal,Rendimiento
0,32,2025-01-20 16:30:00+00:00,11.27,Compra,0.000000
1,120,2025-02-14 14:30:00+00:00,13.47,Venta,0.188296
2,130,2025-02-18 14:30:00+00:00,13.60,Compra,0.000000
3,148,2025-02-21 17:30:00+00:00,13.39,Venta,-0.021135


In [ ]:
df_Stats=df_Resumen[df_Resumen['Rendimiento']!=0]
df_Stats['Rendimiento'].describe().round(4)

,Rendimiento
count,2.0000
mean,0.0836
std,0.1481
min,-0.0211
25%,0.0312
50%,0.0836
75%,0.1359
max,0.1883


In [ ]:
#Crear una columna que me señale la Compra y a que precio
for j in range (0,len(df_Resumen)):
    if df_Resumen.loc[j,'Señal']=='Compra':
        df_Resumen.loc[j,'Señal_Compra']=df_Resumen.loc[j,'Close']
    else:
        df_Resumen.loc[j,'Señal_Compra']=""